#Установка Наташи и скачивание моделей русского языка

In [0]:
!pip install natasha
!wget https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
!tar -xvzf araneum_none_fasttextcbow_300_5_2018.tgz

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 7.1MB 25.6MB/s 
--2020-04-30 15:06:51--  https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2691248108 (2.5G) [application/x-gzip]
Saving to: ‘araneum_none_fasttextcbow_300_5_2018.tgz’

araneum_none_fastte 100%[===================>]   2.51G  25.8MB/s    in 1m 40s  

2020-04-30 15:08:31 (25.6 MB/s) - ‘araneum_none_fasttextcbow_300_5_2018.tgz’ saved [2691248108/2691248108]

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_

#Тестирование Наташи

In [0]:
import natasha

from natasha import NamesExtractor
from natasha import DatesExtractor


text = '''
Король Арту́р — по преданиям, правитель королевства Логрес, легендарный вождь бриттов V—VI веков, разгромивший завоевателей-саксов. 
Самый знаменитый из кельтских героев, центральный герой британского эпоса и многочисленных рыцарских романов. 
Многие историки допускают существование исторического прототипа Артура.
'''
t = '''
Главная достопримечательностью Сан-Франциско построен через пролив с одноименным названием и соединяет городской залив с Тихим океаном. 
Конструкция представляет собой висячий мост с подвешенной проезжей частью, а его длина составляет 1970 м. Работы по возведению моста длились с 1933 по 1937 годы, во времена Великой депрессии и в период становления города от последствий землетрясения 1906 г. Разработчиком проекта был Джозеф Строусс – лучший инженер того времяни по строительству мостов. В конце мая 1937 г. состоялось торжественное открытие самого большого висячей моста в мире на тот момент, 
данный статус мост Золотые ворота сохранял до 1964 г., пока его не обошел нью-йоркский мост Верразано. Уникальный мост признан символом США и является гордостью американского народа.
'''

extractor = NamesExtractor()
matches = extractor(text)
for match in matches:
    print(match.span, match.fact)

extractor2 =  DatesExtractor()
matches = extractor2(t)
for match in matches:
    print(match.span, match.fact)


[53, 59) Name(first='логрес', middle=None, last=None, nick=None)
[309, 315) Name(first='артур', middle=None, last=None, nick=None)
[289, 298) Date(year=1937, month=None, day=None, current_era=True)
[388, 395) Date(year=1906, month=None, day=None, current_era=True)
[500, 511) Date(year=1937, month=5, day=None, current_era=True)
[645, 652) Date(year=1964, month=None, day=None, current_era=True)


#Тестирование pymorphy

In [0]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
mp = morph.parse("фильма")

print('Начальная форма: '+mp[0].normal_form)# Начальная форма
print('Часть речи: '+mp[0].tag.POS ) # Часть речи
print('Одушевлено: '+str(mp[0].tag.animacy == 'anim') ) # Проверка на одушевленность
print('Род: '+mp[0].tag.gender) # Род
print('Падеж: '+mp[0].tag.case) # Падеж

Начальная форма: фильм
Часть речи: NOUN
Одушевлено: False
Род: masc
Падеж: gent


#Инициализациализация модели W2V

In [0]:
import gensim
import numpy as np

model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


#Весовая функция

In [0]:
# пример весовой функции
def Wf(word):
  morph = pymorphy2.MorphAnalyzer()
  p = morph.parse(word)[0]
  if p.tag.POS == 'NOUN':
    if p.tag.case == 'nomn':
      return 5
    else:
      return 3
  elif (p.tag.POS == 'VERB') or (p.tag.POS == 'INFN'):
    return 1.5
  else:
    return 0.7

#Основные функции

In [0]:
import re

# Преобразование слова в вектор
def W2V(word):
  try:
    morph = pymorphy2.MorphAnalyzer()
    mp = morph.parse(word)
    word2 = mp[0].normal_form # Приведение к нормальной форме
    word2 = word2.replace('ё', 'е')
    vect = model[word2] # Вычисление вектора
    return vect
  except:
    return np.zeros(300)

# Преобразование предложения в вектор
def Seq2Vec(seq, isCentr = False): 
  seq = str(seq)
  seq = seq.replace('\n',' ')

  while '  ' in seq:
    seq = seq.replace('  ', ' ')

  reg = re.compile('[^A-zА-яЁё ]')
  seq2 = reg.sub('', seq)
  seqArray = seq2.split(' ')
  vecSeq = np.zeros(300)
  wSum = 0

  for i in seqArray:
    w = Wf(i)
    wSum+=w
    vecSeq += w*W2V(i)

  if isCentr: # Центрирование и нормализация
    vecSeq -= np.mean(vecSeq)
    vecSeq /= np.std(vecSeq)+1e-24
  else:
    vecSeq = vecSeq/wSum#/len(seqArray)

  return vecSeq

# Сравнение текстов
def SimSeq(seq1, seq2, isCentr = False):
  return np.corrcoef(Seq2Vec(seq1, isCentr), Seq2Vec(seq2, isCentr))[0,1] # Получение коэффициента корреляции

#Тест gensim + pymorphy

In [0]:
text = '''
Шаблон регулярного выражения представляет собой специальный язык, 
используемый для представления общего текста, цифр или символов, извлечения текстов, соответствующих этому шаблону.
'''

text1 = '''
Смотрю веселый сериал
'''

text2 = '''
Такие языки как Perl и Ruby фактически поддерживают синтаксис регулярных выражений прямо в собственном языке. 
Python же поддерживает благодаря библиотеки, которую вам нужно импортировать. 
Основное использование регулярных выражений – это сопоставление строк
'''

text3 ='''
Когда-то видел классный фильм
'''

SimSeq(text1, text3)

0.7913359277011551

#Метод ближ. соседа

In [0]:
from sklearn.neighbors import KNeighborsClassifier
 
knn = KNeighborsClassifier(n_neighbors=1)

dataX = np.array([[2,1,1],[1,1,5]])
dataY = np.array(['ghb','ds'])

dataXTest = np.array([[1,1,3]])

knn.fit(dataX, dataY)

knn.predict(dataXTest)[0]

'ds'

#Чат-бот

In [0]:
#Получение векторов из вопросов
def GetDataX(qs):
  retObj = np.zeros(shape=(len(qs),300))
  l = 0

  for i in qs:
    retObj[l] = Seq2Vec(i, True)
    l=l+1
  
  return retObj



# Обучение бота
def Train(knn, qs, ans):
  dataX = GetDataX(qs)
  knn.fit(dataX, ans)

# Ответ
def Answer(knn, q):
  vec = [Seq2Vec(q, True)]
  return knn.predict(vec)[0]


#Данные
qs = ['Привет', 'Сколько стоит дом', 'Как проехать реке', 'До свидания', 'Как жизнь?']
ans = ['Привет!', 'Этот дом стоит $45k', 'Посмотри карту', 'пока', 'норм']

#Инициализация и обучение
knn = KNeighborsClassifier(n_neighbors=1)
Train(knn, qs, ans)



#Тест

In [0]:
Answer(knn, "Здравствуйте")

'Привет!'

In [0]:
Answer(knn, "Как добраться до реки?")

'Посмотри карту'

In [0]:
Answer(knn, "Какова цена этого здания?")

'Этот дом стоит $45k'

In [0]:
while 1:
  text = str(input("Я: "))
  print("Бот:",Answer(knn, text))
  if text == "досвидания": break

Я: как добраться до моря
Бот: Посмотри карту


KeyboardInterrupt: ignored